In [0]:
# κάνουμε update τις βιβλιοθήκες μας
!pip install -U scikit-learn
!pip install --upgrade pandas
!pip install -U scipy
!pip install --upgrade numpy

# Αξιολόγηση ταξινόμησης με ακρίβεια, ανάκληση και F1, μέσοι όροι
Η πιστότητα είναι μια πολύ χρήσιμη και πρακτική μετρική της απόδοσης ενός ταξινομητή. Ωστόσο, δεν αρκεί για μια ολκοληρωμένη μελέτη της απόδοσής του. Είναι τυπικό για όλα τα προβλήματα machine learning να χρησιμοποιούμε διαφορετικές μετρικές για να μελετήσουμε τις ιδιότητες των δεδομένων. 

Καταρχάς, ας ξαναφέρουμε το Iris dataset και τον kNN και ας κάνουμε προβλέψεις με τον default kNN (είναι ακριβώς ο κώδικας από το προηγούμενο notebook σε ένα cell)

In [0]:
# Load Iris and organize our data
from sklearn.datasets import load_iris
data = load_iris()
label_names = data['target_names']
labels = data['target']
feature_names = data['feature_names']
features = data['data']
# Χρησιμοποιούμε τη γνωστή train_test_split για να διαχωρίσουμε σε train και test set
# το (int) όρισμα "random_state" είναι το seed της γεννήτριας τυχαίων αριθμών (αν του δώσουμε τιμή θα παράξει την ίδια σειρά τυχαίων αριθμών)
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.40, random_state=78)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
pred = knn.predict(X_test)

Η βάση για τις μετρικές απόδοσης των ταξινομητών είναι ο πίνακας σύγχυσης (confusion matrix). O πίνακας σύχγησης $C$  είναι τέτοιος ώστε το $C_{i, j}$ είναι ίσο με τα δείγματα που ενώ ανήκουν στην κατηγορία $i$ ταξινομήθηκαν στην κατηγορία  $j$. Για το Iris, στο συγκεκριμένο train/test split, o πίνακας σύγχυσης είναι:

In [5]:
from sklearn.metrics import confusion_matrix
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, pred)
# τυπώνουμε τα labels
print(label_names, "\n")
# τυπώνουμε το confusion matrix
print(cnf_matrix)

['setosa' 'versicolor' 'virginica'] 

[[20  0  0]
 [ 0 22  1]
 [ 0  0 17]]


που σημαίνει ότι 1 δείγμα που άνηκε κανονικά στο είδος versicolor ταξινομήθηκε λανθασμένα στο είδος virginica. Τα στοιχεία της διαγωνίου είναι αληθινά θετικά δείγματα (true positive) της κάθε κλάσης. Για κάθε κλάση $i$ τα στοιχεία της γραμμής $i$ εκτός της διαγωνίου είναι λάνθασμένα αρνητικά δείγματα (false negative) της κλάσης και τα στοιχεία της κολώνας $i$ εκτός της διαγωνίου είναι λανθασμένα θετικά δείγματα (false positive) της κλάσης. 


<figure>
  <center>
  <img src="http://scikit-learn.org/stable/_images/sphx_glr_plot_confusion_matrix_001.png" alt="Confusion Matrix">
   <figcaption>Confusion matrix στο Iris - αλλος ταξινομητής (SVM)</figcaption>
  </center>
</figure>


Σε περίπτωση δυαδικού ταξινομητή $C_{0,0}$ είναι τα αληθινά αρνητικά δείγματα (true negative, η κλάση 0 θεωρείται η αρνητική), $C_{1,0}$ είναι τα λανθασμένα αρνητικά δείγματα, $C_{1,1}$ τα αληθινά θετικά δείγματα και $C_{0,1}$ τα λανθασμένα θετικά. Για παράδειγμα:

In [6]:
# παράδειγμα confusion matrix σε δυαδική ταξινόμηση
tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()
print(tn, fp, fn, tp)

0 2 1 1


Συχνά στην δυαδική ταξινόμηση θεωρούμε θετική την πιο σπάνια κλάση ή το φαινόμενο προς εντοπισμό (πχ διαβητικός). 

![CMMulti2](http://image.ntua.gr/~tpar/dataminingimg/confusionMatrix2small.PNG)

Ορίζουμε:
Ακρίβεια -Precision- ($P$) είναι ο λόγος των true positives ($T_p$) ως προς τον αριθμό των true positives συν τον αριθμό των false positives ($F_p$).
$$P = \frac{T_p}{T_p+F_p}$$
Ανάκληση -Recall- ($R$) είναι ο λόγος των true positives ($T_p$) ως προς τον αριθμό των true positives συν τον αριθμό των false negatives ($F_n$).
$$R = \frac{T_p}{T_p + F_n}$$
Συχνά χρησιμοποιούμε και το ($F_1$) score, το οποίο είναι ο αρμονικός μέσος της ακρίβειας και της ανάκλησης.
$$F1 = 2\frac{P \times R}{P+R}$$
Ιδανικά θέλουμε και υψηλή ακρίβεια και υψηλή ανάκληση, ωστόσο μεταξύ της ακρίβειας και της ανάκλησης υπάρχει γενικά trade-off. Στην οριακή περίπτωση του ταξινομητή που επιστρέφει σταθερά μόνο τη θετική κλάση για παράδειγμα, η ανάκληση θα είναι 1 αλλά η ακρίβεια θα έχει τη μικρότερη δυνατή τιμή της. Γενικά, κατεβάζοντας το κατώφλι της απόφασης του ταξινομητή, αυξάνουμε την ανάκληση και μειώνουμε την ακρίβεια και αντιστρόφως.

Στην πράξη και ειδικά σε μη ισορροπημένα datasets χρησιμοποιούμε την ακρίβεια, ανάκληση και το F1 πιο συχνά από την πιστότητα. Επίσης, ανάλογα την εφαρμογή μπορεί να μας ενδιαφέρει περισσότερο ένα συγκεκριμένο metric, πχ η ανάκληση (πχ στη διάγνωση μιας ασθένειας) ή η ακρίβεια (πχ σε μια οικονομική απόφαση).

In [7]:
from sklearn.metrics import precision_recall_fscore_support

# εκτυπώνουμε 4 πίνακες, precision, recall, F1 και support. Support είναι ο συνολικός αριθμός προβλέψεων σε κάθε κλάση
# το πρώτο στοιχείο του κάθε πίνακα είναι η κλάση setosa, το δεύτερο η versicolor και το τρίτο η virginica
print(precision_recall_fscore_support(y_test, pred, average=None), "\n")

# εκτυπώνουμε τa precision, recall και F1 λαμβάνοντας υπόψη συνολικά (αθροίζοντας εκτός κλάσεων) τα δείγματα (average = micro).
print(precision_recall_fscore_support(y_test, pred, average='micro'), "\n")

# εκτυπώνουμε το μέσο όρο των precision, recall και F1 θεωρόντας ότι οι κλάσεις έχουν το ίδιο βάρος (average = macro)
print(precision_recall_fscore_support(y_test, pred, average='macro'), "\n")

# εκτυπώνουμε τa precision, recall και F1 λαμβάνοντας. Με average = weighted κάθε κλάση μετρά στο μέσο όρο ανάλογα με το support της.
print(precision_recall_fscore_support(y_test, pred, average='weighted'), "\n")

# η classification_report τυπώνει πιο ωραία οπτικά σε string τα αποτελέσματα
# πρώτα για κάθε κλάση και μετά με μέσους όρους
from sklearn.metrics import classification_report
print(classification_report(y_test, pred, target_names=label_names))

(array([1.        , 1.        , 0.94444444]), array([1.        , 0.95652174, 1.        ]), array([1.        , 0.97777778, 0.97142857]), array([20, 23, 17])) 

(0.9833333333333333, 0.9833333333333333, 0.9833333333333333, None) 

(0.9814814814814815, 0.9855072463768115, 0.9830687830687831, None) 

(0.9842592592592593, 0.9833333333333333, 0.9833862433862434, None) 

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        20
  versicolor       1.00      0.96      0.98        23
   virginica       0.94      1.00      0.97        17

    accuracy                           0.98        60
   macro avg       0.98      0.99      0.98        60
weighted avg       0.98      0.98      0.98        60



Documentation από το scikit σχετικά με την αξιολόγηση των μοντέλων:

* [sklearn.metrics.precision_recall_fscore_support](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html)
* [Model evaluation: quantifying the quality of predictions](http://scikit-learn.org/stable/modules/model_evaluation.html#multiclass-and-multilabel-classification)
* [Multiclass and multilabel classification](http://scikit-learn.org/stable/modules/model_evaluation.html#multiclass-and-multilabel-classification)

Σημειώστε ότι όταν η classification report μέχρι το version 0.19 του scikit τυπώνει μόνο ένα μέσο όρο, τον weighted. (σχετική συζήτηση [εδώ](https://stackoverflow.com/questions/23914472/strange-f1-score-result-using-scikit-learn)).

# Άσκηση: Σύγκριση του Gausian Naive Bayes και του kNN στo Pima Indians Diabetes dataset
![1889 Photograph shows half-length portrait of two Pima Indians, facing front, wearing bead necklaces.](https://i.pinimg.com/236x/60/05/76/600576905d4ad5bb1a9c3e3387b397ca--pima-indians-native-american-indians.jpg "1889 Photograph shows half-length portrait of two Pima Indians, facing front, wearing bead necklaces.")

Διαβάστε το ["pima-indians-diabetes.data.csv"](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv) σε ένα numpy array data και ξεχωρίστε features και labels.

Για 40% test set: 
1. υπολογίστε την πρόβλεψη του Gaussian Naive Bayes με τη μέθοδο predict(). 
2. πάρτε τις προβλέψεις ενός kNN με k=5
3. Για έναν ταξινομητή kNN, με 3-fold cross validation και με μετρική 'f1_weighted' υπολογίστε το βέλτιστο k στο train set (maximum k=50). 
4. εκτυπώστε με την "classification_report" τα precision, recall, f1, support για τον NB, τον kNN με k=5 και με το k που έχει προκύψει από cross validation.
5. Κάντε 3 runs και αποθηκεύστε σε ένα κελί markdown το average F1 του non optimized και του optimized kNN. Πόσο % έχει βελτιβωθεί η επίδοσή του; 

hint: Για τη δημιουργία πινάκων σε markdown μπορείτε να χρησιμοποιείτε ένα [markdown table generator](https://www.tablesgenerator.com/markdown_tables)